## End-to-End Scenario: Analyze the Cash Flow of an Investment on a New Product

TI HA DB ML China - SAP HANA PAL Team

2020/06/18

In this end-to-end scenario, we wish to do an analysis of the cash flow of an investment required to create a new product. Projected estimates are given for the product revenue, product costs, overheads, and capital investment for each year of the analysis, from which the cash flow can be calculated. For capital investment appraisal the cash flows are summed for each year and discounted for future values, in other words the net present value of the cash flow is derived as a single value measuring the benefit of the investment.

The projected estimates are single point estimates of each data point and the analysis provides a single point value of project net present value (NPV). This is referred to as deterministic modeling, which is in contrast to probabilistic modeling whereby we examine the probability of outcomes, for example, what is the probability of a NPV greater than zero. Probabilistic modeling is also called Monte Carlo Simulation.

Monte Carlo Simulation is used in our example to estimate the net present value (NPV) of the investment. The equations used in the simulation are:

For each year i=0, 1, …, k

Product margin(i) = product revenue(i) – product cost(i)

Total profit(i) = product margin(i) – overhead(i)

Cash flow(i) = total profit(i) – capital investment(i)

Suppose the simulation covers k years’ time periods and the discount rate is r, the net present value of the investment is defined as:

![](npv.png)

## 1. Technology Background
Monte Carlo Simulation is a computational algorithm that repeatedly generates random samples to compute numerical results based on a formula or model in order to obtain the unknown probability distribution of an event or outcome.

In hana_ml, the Random Distribution Sampling, Distribution Fitting, and Cumulative Distribution algorithms may be used for Monte Carlo Simulation.

## 2. Implementation Steps

**Setup the Connection to SAP HANA**

First, create a connetion to SAP HANA. To create a such connection, a config file, config/e2edata.ini is used to control the connection parameters.A sample section in the config file is shown below which includes HANA url, port, user and password information.<br>

###################<br>
[hana]<br>
url=host-url<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
###################<br>


In [ ]:
from hana_ml.dataframe import ConnectionContext
from hana_ml.algorithms.pal.utility import Settings
url, port, user, pwd = Settings.load_config("../../../config/e2edata.ini")
connection_context = ConnectionContext(url, port, user, pwd)

Connection functions samples:

In [ ]:
print(connection_context.connection.isconnected())

### Step 2: Dataset Generation

Input the given estimates (single point deterministic values) for product revenue, product costs, overheads, and capital investment. In this example, the time periods are 5 (from year 1 to year 5).

The probability distribution for each variable is assumed as follows:

Product Revenue:
Normal distribution and the mean and standard deviation are listed in the following table.

Product Costs:
Normal distribution and the mean and standard deviation are listed in the following table.

Overheads:
Uniform distribution and min and max values are listed in the following table.

Capital Investment (for year 1 and year 2)
Gamma distribution and shape and scale values are listed in the following table.

![](table.png)

 	 	 
Run the Random Distribution Sampling algorithm for each variable and generate 5,000 sample sets. The number of sample sets is a choice for the analysis. The larger the value then the more smooth the output distribution and the closer it will be to a normal distribution.

**The first year**

In [ ]:
from hana_ml.algorithms.pal.random import normal, uniform
cursor = connection_context.connection.cursor()

number = 10
    
# Product Revenue:
revenue_1 = normal (connection_context, num_random = number, mean = 0, sigma = 0.01, seed = 0)
revenue_1 = revenue_1.rename_columns(['ID', 'RANDOM'])
try:
     cursor.execute("DROP TABLE REVENUE_1_TBL;")
except:
     pass
revenue_1.save('REVENUE_1_TBL')

# Product Costs:
cost_1 = normal (connection_context, num_random = number, mean = 1000, sigma = 75, seed = 0)
cost_1 = cost_1.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE COST_1_TBL;")
except:
     pass    
cost_1.save('COST_1_TBL')

# Overheads:
overheads_1 = uniform(connection_context, num_random = number, low = 1400, high = 1500, seed = 0)
overheads_1 = overheads_1.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE OVERHEAD_1_TBL;")
except:
     pass
overheads_1.save('OVERHEAD_1_TBL')

# Capital Investment (for year 1 and year 2)
investment_1 = normal (connection_context, num_random = number, mean = 10000, sigma = 500, seed = 0)
investment_1 = investment_1.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE INVESTMENT_1_TBL;")
except:
     pass
investment_1.save('INVESTMENT_1_TBL')

try:
    cursor.execute("DROP TABLE PAL_CASHFLOW_YEAR1;")
except:
     pass
cursor.execute("CREATE COLUMN TABLE PAL_CASHFLOW_YEAR1(ID INTEGER, CASH DOUBLE);")
sql = "INSERT INTO PAL_CASHFLOW_YEAR1" + " SELECT REVENUE_1_TBL.ID, REVENUE_1_TBL.RANDOM - COST_1_TBL.RANDOM - OVERHEAD_1_TBL.RANDOM - INVESTMENT_1_TBL.RANDOM FROM REVENUE_1_TBL" + " LEFT JOIN COST_1_TBL ON REVENUE_1_TBL.ID = COST_1_TBL.ID" +" LEFT JOIN OVERHEAD_1_TBL ON REVENUE_1_TBL.ID = OVERHEAD_1_TBL.ID " + " LEFT JOIN INVESTMENT_1_TBL ON REVENUE_1_TBL.ID = INVESTMENT_1_TBL.ID;"
cursor.execute(sql) 


print(connection_context.table('REVENUE_1_TBL').head(5).collect())
print(connection_context.table('COST_1_TBL').head(5).collect())
print(connection_context.table('OVERHEAD_1_TBL').head(5).collect())
print(connection_context.table('INVESTMENT_1_TBL').head(5).collect())
print(connection_context.table('PAL_CASHFLOW_YEAR1').head(5).collect())

**The second year**

In [3]:
# Product Revenue:
revenue_2 = normal (connection_context, num_random = number, mean = 3000, sigma = 300, seed = 0)
revenue_2 = revenue_2.rename_columns(['ID', 'RANDOM'])
try:
     cursor.execute("DROP TABLE REVENUE_2_TBL;")
except:
     pass
revenue_2.save('REVENUE_2_TBL')

# Product Costs:
cost_2 = normal (connection_context, num_random = number, mean = 1000, sigma = 75, seed = 0)
cost_2 = cost_2.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE COST_2_TBL;")
except:
     pass    
cost_2.save('COST_2_TBL')

# Overheads:
overheads_2 = uniform(connection_context, num_random = number, low = 1800, high = 2200, seed = 0)
overheads_2 = overheads_2.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE OVERHEAD_2_TBL;")
except:
     pass
overheads_2.save('OVERHEAD_2_TBL')

# Capital Investment (for year 1 and year 2)
investment_2 = normal (connection_context, num_random = number, mean = 2000, sigma = 100, seed = 0)
investment_2 = investment_2.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE INVESTMENT_2_TBL;")
except:
     pass
investment_2.save('INVESTMENT_2_TBL')

try:
    cursor.execute("DROP TABLE PAL_CASHFLOW_YEAR2;")
except:
     pass
cursor.execute("CREATE COLUMN TABLE PAL_CASHFLOW_YEAR2(ID INTEGER, CASH DOUBLE);")
sql = "INSERT INTO PAL_CASHFLOW_YEAR2" + " SELECT REVENUE_2_TBL.ID, REVENUE_2_TBL.RANDOM - COST_2_TBL.RANDOM - OVERHEAD_2_TBL.RANDOM - INVESTMENT_2_TBL.RANDOM FROM REVENUE_2_TBL" + " LEFT JOIN COST_2_TBL ON REVENUE_2_TBL.ID = COST_2_TBL.ID" +" LEFT JOIN OVERHEAD_2_TBL ON REVENUE_2_TBL.ID = OVERHEAD_2_TBL.ID " + " LEFT JOIN INVESTMENT_2_TBL ON REVENUE_2_TBL.ID = INVESTMENT_2_TBL.ID;"
cursor.execute(sql) 


print(connection_context.table('REVENUE_2_TBL').head(5).collect())
print(connection_context.table('COST_2_TBL').head(5).collect())
print(connection_context.table('OVERHEAD_2_TBL').head(5).collect())
print(connection_context.table('INVESTMENT_2_TBL').head(5).collect())
print(connection_context.table('PAL_CASHFLOW_YEAR2').head(5).collect())


   ID       RANDOM
0   0  1048.534062
1   1   893.906402
2   2   986.954984
3   3  1057.956127
4   4  1039.476038
   ID       RANDOM
0   0  1865.758594
1   1  2137.236783
2   2  1986.334537
3   3  2018.941237
4   4  2032.311356
   ID       RANDOM
0   0  1965.999822
1   1  2000.922336
2   2  1951.856289
3   3  1984.909012
4   4  1998.743250
   ID         CASH
0   0 -1755.374232
1   1 -2339.244330
2   2 -1700.007533
3   3 -1801.603240
4   4 -1886.999862


In [ ]:
# Product Revenue:
revenue_3 = normal (connection_context, num_random = number, mean = 8000, sigma = 800, seed = 0)
revenue_3 = revenue_3.rename_columns(['ID', 'RANDOM'])
try:
     cursor.execute("DROP TABLE REVENUE_3_TBL;")
except:
     pass
revenue_3.save('REVENUE_3_TBL')

# Product Costs:
cost_3 = normal (connection_context, num_random = number, mean = 2500, sigma = 187.5, seed = 0)
cost_3 = cost_3.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE COST_3_TBL;")
except:
     pass    
cost_3.save('COST_3_TBL')

# Overheads:
overheads_3 = uniform(connection_context, num_random = number, low = 2200, high = 2800, seed = 0)
overheads_3 = overheads_3.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE OVERHEAD_3_TBL;")
except:
     pass
overheads_3.save('OVERHEAD_3_TBL')

try:
    cursor.execute("DROP TABLE PAL_CASHFLOW_YEAR3;")
except:
     pass
cursor.execute("CREATE COLUMN TABLE PAL_CASHFLOW_YEAR3(ID INTEGER, CASH DOUBLE);")
sql = "INSERT INTO PAL_CASHFLOW_YEAR3" + " SELECT REVENUE_3_TBL.ID, REVENUE_3_TBL.RANDOM - COST_3_TBL.RANDOM - OVERHEAD_3_TBL.RANDOM FROM REVENUE_3_TBL" + " LEFT JOIN COST_3_TBL ON REVENUE_3_TBL.ID = COST_3_TBL.ID" + " LEFT JOIN OVERHEAD_3_TBL ON REVENUE_3_TBL.ID = OVERHEAD_3_TBL.ID "
cursor.execute(sql) 


print(connection_context.table('REVENUE_3_TBL').head(5).collect())
print(connection_context.table('COST_3_TBL').head(5).collect())
print(connection_context.table('OVERHEAD_3_TBL').head(5).collect())
print(connection_context.table('PAL_CASHFLOW_YEAR3').head(5).collect())

In [ ]:
# Product Revenue:
revenue_4 = normal (connection_context, num_random = number, mean = 18000, sigma = 1800, seed = 0)
revenue_4 = revenue_4.rename_columns(['ID', 'RANDOM'])
try:
     cursor.execute("DROP TABLE REVENUE_4_TBL;")
except:
     pass
revenue_4.save('REVENUE_4_TBL')

# Product Costs:
cost_4 = normal (connection_context, num_random = number, mean = 7000, sigma = 525, seed = 0)
cost_4 = cost_4.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE COST_4_TBL;")
except:
     pass    
cost_4.save('COST_4_TBL')

# Overheads:
overheads_4 = uniform(connection_context, num_random = number, low = 2600, high = 3400, seed = 0)
overheads_4 = overheads_4.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE OVERHEAD_4_TBL;")
except:
     pass
overheads_4.save('OVERHEAD_4_TBL')

try:
    cursor.execute("DROP TABLE PAL_CASHFLOW_YEAR4;")
except:
     pass
cursor.execute("CREATE COLUMN TABLE PAL_CASHFLOW_YEAR4(ID INTEGER, CASH DOUBLE);")
sql = "INSERT INTO PAL_CASHFLOW_YEAR4" + " SELECT REVENUE_4_TBL.ID, REVENUE_4_TBL.RANDOM - COST_4_TBL.RANDOM - OVERHEAD_4_TBL.RANDOM FROM REVENUE_4_TBL" + " LEFT JOIN COST_4_TBL ON REVENUE_4_TBL.ID = COST_4_TBL.ID" +" LEFT JOIN OVERHEAD_4_TBL ON REVENUE_4_TBL.ID = OVERHEAD_4_TBL.ID "
cursor.execute(sql) 


print(connection_context.table('REVENUE_4_TBL').head(5).collect())
print(connection_context.table('COST_4_TBL').head(5).collect())
print(connection_context.table('OVERHEAD_4_TBL').head(5).collect())
print(connection_context.table('PAL_CASHFLOW_YEAR4').head(5).collect())

In [ ]:
# Product Revenue:
revenue_5 = normal (connection_context, num_random = number, mean = 30000, sigma = 3000, seed = 0)
revenue_5 = revenue_5.rename_columns(['ID', 'RANDOM'])
try:
     cursor.execute("DROP TABLE REVENUE_5_TBL;")
except:
     pass
revenue_5.save('REVENUE_5_TBL')

# Product Costs:
cost_5 = normal (connection_context, num_random = number, mean = 5000, sigma = 750, seed = 0)
cost_5 = cost_5.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE COST_5_TBL;")
except:
     pass    
cost_5.save('COST_5_TBL')

# Overheads:
overheads_5 = uniform(connection_context, num_random = number, low = 3000, high = 4000, seed = 0)
overheads_5 = overheads_5.rename_columns(['ID', 'RANDOM'])
try:
    cursor.execute("DROP TABLE OVERHEAD_5_TBL;")
except:
     pass
overheads_5.save('OVERHEAD_5_TBL')

try:
    cursor.execute("DROP TABLE PAL_CASHFLOW_YEAR5;")
except:
     pass
cursor.execute("CREATE COLUMN TABLE PAL_CASHFLOW_YEAR5(ID INTEGER, CASH DOUBLE);")
sql = "INSERT INTO PAL_CASHFLOW_YEAR5" + " SELECT REVENUE_5_TBL.ID, REVENUE_5_TBL.RANDOM - COST_5_TBL.RANDOM - OVERHEAD_5_TBL.RANDOM FROM REVENUE_5_TBL" + " LEFT JOIN COST_5_TBL ON REVENUE_5_TBL.ID = COST_5_TBL.ID" +" LEFT JOIN OVERHEAD_5_TBL ON REVENUE_5_TBL.ID = OVERHEAD_5_TBL.ID "
cursor.execute(sql) 


print(connection_context.table('REVENUE_5_TBL').head(5).collect())
print(connection_context.table('COST_5_TBL').head(5).collect())
print(connection_context.table('OVERHEAD_5_TBL').head(5).collect())
print(connection_context.table('PAL_CASHFLOW_YEAR5').head(5).collect())

### Step 3: Net Present Value Calculation

Calculate the net present value of the investment by the following equation for each sampling.

In [ ]:
try:
    cursor.execute("DROP TABLE NPV;")
except:
    pass
cursor.execute("CREATE COLUMN TABLE NPV ( NPVALUE DOUBLE);")
cursor.execute("INSERT INTO NPV SELECT PAL_CASHFLOW_YEAR1.CASH + PAL_CASHFLOW_YEAR2.CASH/1.05 + PAL_CASHFLOW_YEAR3.CASH/POWER(1.05,2) +  PAL_CASHFLOW_YEAR4.CASH/POWER(1.05,3) + PAL_CASHFLOW_YEAR5.CASH/POWER(1.05,4) FROM PAL_CASHFLOW_YEAR1 LEFT JOIN PAL_CASHFLOW_YEAR2 ON PAL_CASHFLOW_YEAR1.ID = PAL_CASHFLOW_YEAR2.ID LEFT JOIN PAL_CASHFLOW_YEAR3 ON PAL_CASHFLOW_YEAR1.ID = PAL_CASHFLOW_YEAR3.ID LEFT JOIN PAL_CASHFLOW_YEAR4 ON PAL_CASHFLOW_YEAR1.ID = PAL_CASHFLOW_YEAR4.ID LEFT JOIN PAL_CASHFLOW_YEAR5 ON PAL_CASHFLOW_YEAR1.ID = PAL_CASHFLOW_YEAR5.ID;")

print(connection_context.table('NPV').head(5).collect())

### Step 4: Model Fitting

Plot the distribution of the net present value of the investment and run Distribution Fitting to fit a normal distribution to the NPV of the investment as. (The Central Limit theorem states that the output distribution will be a normal distribution.)

In [ ]:
from hana_ml.algorithms.pal.stats import distribution_fit
npv = connection_context.table('NPV')
result_npv = distribution_fit(data = npv, 
                              distr_type = "normal",
                              optimal_method = "maximum_likelihood", 
                              censored=False)
print(result_npv[0].collect())
print(result_npv[1].collect())

### Step 5: CDF  

According to the fitted model, run the Cumulative Distribution function to obtain the probability of having an NPV of investment smaller than or equal to a given NPV of the investment.

Prepare the data:

In [ ]:
cursor = connection_context.connection.cursor()
try:
    cursor.execute("DROP TABLE PAL_DISTRPROB_DATA_TBL")
except:
    pass
cursor.execute('CREATE COLUMN TABLE PAL_DISTRPROB_DATA_TBL (\"DATACOL\" DOUBLE);')
values = [(7000,),(8000,),(9000,),(10000,),(11000,)]
try:
    cursor.executemany("INSERT INTO " +
                       "{} VALUES ({})".format('PAL_DISTRPROB_DATA_TBL',
                       ', '.join(['?']*len(values[0]))), values)
    connection_context.connection.commit()
finally:
    cursor.close()
distri_prob_df = connection_context.table("PAL_DISTRPROB_DATA_TBL")

print(distri_prob_df.collect())

Invoke CDF and fetch the mean and variance from result.npv:

In [ ]:
from hana_ml.algorithms.pal.stats import cdf
mean = float(result_npv[0].collect()['VALUE'][1])
sd = float(result_npv[0].collect()['VALUE'][2])
variance = sd*sd
distr_info = {'name' : 'normal', 'mean' :  mean, 'variance': variance}
result = cdf(distri_prob_df, distr_info, complementary=False)
print(result.collect())

## Close HANA Connection

In [ ]:
tbls = ("REVENUE_1_TBL","REVENUE_2_TBL", "REVENUE_3_TBL", 
             "REVENUE_4_TBL", "REVENUE_5_TBL",
             "COST_1_TBL","COST_2_TBL", "COST_3_TBL", 
             "COST_4_TBL", "COST_5_TBL",
             "OVERHEADS_1_TBL","OVERHEADS_2_TBL", "OVERHEADS_3_TBL", 
             "OVERHEADS_4_TBL", "OVERHEADS_5_TBL",
             "INVESTMENT_1_TBL","INVESTMENT_2_TBL",
             "PAL_CASHFLOW_YEAR1","PAL_CASHFLOW_YEAR2", "PAL_CASHFLOW_YEAR3", 
             "PAL_CASHFLOW_YEAR4", "PAL_CASHFLOW_YEAR5",
             "NPV", "PAL_DISTRPROB_DATA_TBL")
for table in tbls:
    try:
        cursor.execute(table) 
    except:
        pass
connection_context.close()